In [1]:
import json
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report

# 1. Loading Dataset File

## 1.1 original data

In [2]:
# load tweets as dictionary
with open('./Twitter_User_Handles_labeled_tweets.json', 'r', encoding='utf8') as fp:
    tweets = json.load(fp)

In [3]:
# # load profile as list
with open('./User demo profiles.json', 'r', encoding='utf8') as fp:
    profile = json.load(fp)

# translate profile to a dictionary
key_list = []
for dic in profile:
    key_list.append(dic['id'])

profile = dict(zip(key_list, profile))

In [4]:
dataset = pd.read_csv('labeled_users.csv')

In [5]:
dataset

,user_id,is_female,year_born,race
0,1.248800e+04,0.0,1980.0,4.0
1,7.197030e+05,0.0,1985.0,4.0
2,7.221530e+05,1.0,1973.0,3.0
3,7.490030e+05,0.0,1982.0,5.0
4,7.555310e+05,0.0,1982.0,4.0
...,...,...,...,...
4127,7.060000e+17,0.0,1997.0,4.0
4128,7.060000e+17,0.0,1991.0,4.0
4129,7.180000e+17,1.0,1988.0,2.0
4130,7.190000e+17,1.0,1986.0,1.0


## 1.2 data 1145

In [6]:
with open('./labeled_users_1145/tweets.json', 'r', encoding='utf8') as fp:
    tweets_1145 = json.load(fp)

In [7]:
tweets_1145['_____zac_____']

['@AdvoBarryRoux @GetVidBot',
 "The owner of drip doesn't even have 100 mill, drip as a company aint got 100 mill cash, even if we put together they assets that company hardly values at 100 mill.\n\n#bolt #casper #drip #legacydeal",
 'even Lekau the owner of Drip was saying that it was Casspers idea that very same morning that they say it\'s a "100 million  rand deal". \'Cassper is the typa guy who\'d rent rolls Royce\'s n stuff just to show off". I guess you must fake it till you make it.',
 '@casspernyovest is cappin  that "R100m" figure is so inflated, the deal ain\'t worth 100mil, they\'ll probably make 100 mil in some years 2 come, nt anytime soon, so he basically lying 2 us. He should\'ve just said hes signed the #LegacyDeal wit drip &amp; not give it a fake value. Bolt',
 "I want a recipe from @JBscotchSA for #JBLemonade! If you're looking for a change up to your usual drink then Tweet a button below to get a recipe! 😎 #JBIyazehelela #DareToBeRare",
 "@AdvoBarryRoux Am I the onl

In [8]:
dataset_1145 = pd.read_csv('./labeled_users_1145/labeled_users.csv', encoding='utf-8', engine='python')
dataset_1145

,Unnamed: 0,Unnamed: 0.1,screen_name,user_id,lang,name,location,description,protected,followers_count,...,profile_background_url,profile_image_url,user.name,num.tweets.used.Lexicon.prediction,Lexicon.age.prediction,Lexicon.gender.prediction..index.,lexicon.gender.prediction,human.labeled.gender,human.labeled.age,age
0,0,1,_____zac_____,4.614412e+08,en,zac ¢,"Maryland, USA",_____Û___È_Ü´Ù,False,208,...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1226134911...,@_____zac_____,100.0,27.652434,-1.457167,M,NaN,23.0,1.0
1,1,2,___aleia,7.650000e+17,en,_æ___ dad ___æ_,"Ohio, USA",BLACK. LIVES. MATTER.,False,466,...,NaN,http://pbs.twimg.com/profile_images/1271280679...,@___aleia,100.0,24.111464,0.985713,F,NaN,19.0,0.0
2,3,4,___schaeffer___,1.257110e+09,en,Brenden Schaeffer,The Lou,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,False,811,...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1268044218...,@___schaeffer___,59.0,35.518352,-3.591586,M,NaN,22.0,0.0
3,8,9,__drewc,1.050000e+18,en,drew,"New York, USA",_öÂ_öé _öÂ_ö_ _ç´Ù È,None,None,...,None,None,None,NaN,NaN,NaN,None,None,NaN,NaN
4,26 #NewYork,False,27,1.280000e+02,181,113,10/17/18 4:12,False,https://pbs.twimg.com/profile_banners/10524118...,http://abs.twimg.com/images/themes/theme1/bg.png,...,NaN,26.0,1.0,NaN,NaN,NaN,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,3271,3272,ZoeBerrier,9.020000e+17,en,Zoâ _êâ,"Millersville, PA",MU 2021 (she/her),None,None,...,None,None,None,NaN,NaN,NaN,None,None,NaN,NaN
1225,Queen of putting lipstick on in the reflection...,False,94,2.980000e+02,858,4428,8/28/17 21:03,False,https://pbs.twimg.com/profile_banners/90227535...,NaN,...,F,20.0,0.0,NaN,NaN,NaN,None,None,NaN,NaN
1226,3272,3273,ZoeCalamaco,3.214954e+09,no,Zoe _,San Angelo tx/ aspermont tx,Angelo state,False,475,...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1277399838...,@ZoeCalamaco,100.0,21.348766,1.939069,F,F,21.0,0.0
1227,3274,3275,ZoPeachy,9.890000e+17,en,Zobella Thee Alpha __ê_____´Ù__ ...,"New England/Boston, MA",Harlot for hire. FinDom. 27. Nonbinary. they/t...,False,396,...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1153648270...,@ZoPeachy,100.0,34.894953,2.058720,F,NaN,27.0,1.0


# 2. Pre-processing

In [9]:
print(dataset['user_id'].isna().sum())
print(dataset['is_female'].isna().sum())
print(dataset['year_born'].isna().sum())
print(dataset['race'].isna().sum())

0
63
48
40


In [10]:
print(dataset_1145['screen_name'].isna().sum())
print(dataset_1145['age'].isna().sum())

35
133


In [25]:
# delete username, http links and https links in twitters
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"", r"http://[\w./]+":"", r"https://[\w./]+":""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t

# match users with their tweets
def match_sentences(user_id, tweets):
    if str(user_id) in tweets:
        sentences = ".".join(tweets[str(user_id)])
        return preprocess(sentences)
    else:
        return None

# get features from profile    
def add_feature_from_profile(user_id, feature_name):
    if user_id in profile:
        return profile[user_id][feature_name]
    else:
        return None

# match image path with user
def create_img_path(num):
    if num == 0:
        return None
    else:
        return 'labeled_users_1145/profile pics test/' + str(num - 1) + '.jpeg'
    
    
# pre-processing for original dataset
def raw_data_pre_processing(raw_data, tweets):
    # delete rows with nan values
    new_dataset = raw_data.dropna(axis=0, how='any')
    new_dataset = new_dataset.astype(np.int64)
    
    # delete multiracial data
    new_dataset = new_dataset[new_dataset['race'] != 5]
    
    # translate year label
    # if age >= 21, age value=1
    new_dataset['age'] = ((2021 - new_dataset[['year_born']]) >= 21).astype(np.int64)
    # drop 'is_female' and 'year_born'
    new_dataset = new_dataset.drop(columns=['is_female', 'year_born']).reset_index(drop=True)
    # add twitters as feature
    new_dataset['tweets'] = new_dataset['user_id'].apply(match_sentences, tweets=tweets)
    # drop users who do not have twitters
    new_dataset = new_dataset.replace(to_replace='None', value=np.nan).dropna().reset_index(drop=True)
    # get features from profile
    features = ['name', 'screen_name', 'description', 'lang', 'img_path']
    for feature in features:
        new_dataset[feature] = new_dataset['user_id'].apply(add_feature_from_profile, feature_name=feature)
    
    
    return new_dataset


# pre-processing for dataset_1145
def raw_data_pre_processing_for_1145(raw_data, tweets):
    # remain columns = ['screen_name', 'age']
    new_dataset = raw_data[['age', 'screen_name', 'name', 'lang', 'description', 'Unnamed: 0.1']]
    # delete rows with nan values
    new_dataset = new_dataset.dropna(axis=0, how='any')
    new_dataset['age'] = new_dataset['age'].astype(np.int64)
    new_dataset['tweets'] = new_dataset['screen_name'].apply(match_sentences, tweets=tweets)
    # match image path
    new_dataset['img_path'] = pd.to_numeric(new_dataset['Unnamed: 0.1'], errors='coerce').fillna('0').astype(np.int64)
    new_dataset['img_path'] = new_dataset['img_path'].apply(create_img_path)
    new_dataset = new_dataset.drop(columns=['Unnamed: 0.1'])
    new_dataset = new_dataset.replace(to_replace='None', value=np.nan).dropna().reset_index(drop=True)
    
    return new_dataset

def augment_data(train_data):
    num_while = len(train_data[train_data['race'] == 4])
    for i in [1,2,3]:
        data = train_data[train_data['race'] == i]
        nums = int(num_while/len(data))-1
        for _ in range(nums):
            train_data = train_data.append(data)
    return train_data

def merge_tweets_and_description(dataset):
    # add description to twitter
    dataset['tweets'] = dataset['description'].apply(preprocess) + dataset['tweets']
    return dataset

In [26]:
# raw data pre-processing
new_dataset = raw_data_pre_processing(dataset, tweets)
new_dataset_1145 = raw_data_pre_processing_for_1145(dataset_1145, tweets_1145)

In [13]:
# save clean dataset
new_dataset.to_csv('./clean_dataset.csv', index=False)
new_dataset_1145.to_csv('./clean_dataset_1145.csv', index=False)

## 2.1 oringinal data after pre-processing

In [27]:
new_dataset

,user_id,race,age,tweets,name,screen_name,description,lang,img_path
0,12488,4,1,"YKAR, a futuristic sans serif font by #Freeb...",Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NA,profile pics/60147.jpeg
1,719703,4,1,"In other words, it’s good news about the vacci...",Tomato 😷,Tomato,🇭🇰Rise Up!,NA,profile pics/60148.jpeg
2,811618,3,1,Blah blah blah.I think RAPE is worst! RT Bein...,Mr.O,Putanginamo,http://t.co/UfipjuQ2Mw is a blog and talk show...,NA,profile pics/60152.jpeg
3,822540,4,1,❤️ 🙏. bonk. #FFXIV400kSweepstakes. Nice. Ed...,parker,parker,gotta go fast,NA,profile pics/60153.jpeg
4,865071,4,1,How about pizza dipped in water 🤦🏻‍♂️.Day 21 ...,Kevin Jones,kevinj,,NA,profile pics/60154.jpeg
...,...,...,...,...,...,...,...,...,...
3046,4892221799,4,1,What a beautiful thing to see this ...,Amanda,BohoGrlNxtDoor,Just trying to stand out in the crowd. Which i...,NA,profile pics/64265.jpeg
3047,4895390642,2,1,Inspired Attempt Tour: like when you first try...,Dalton's Eyeliner,FlyAwayEyeliner,"I'm Dalton's eyeliner, who are you? Are you Da...",NA,profile pics/64267.jpeg
3048,4921995243,4,1,yo,saved,memelady,,NA,profile pics/64269.jpeg
3049,4924158634,4,1,Those are amazing 😆.Looked over to see both b...,Winter da CoffeeCat,WinterStar21,"Big fan of cats, coffee, horror stuff and anim...",NA,profile pics/64270.jpeg


In [28]:
new_dataset['race'].value_counts()

4    2482
1     290
2     181
3      98
Name: race, dtype: int64

## 2.2 dataset_1145 after pre-processing

In [60]:
new_dataset_1145

,age,screen_name,name,lang,description,tweets,img_path
0,1,_____zac_____,zac ¢,en,_____Û___È_Ü´Ù,".The owner of drip doesnt even have 100 mill,...",labeled_users_1145/profile pics test/0.jpeg
1,0,___aleia,_æ___ dad ___æ_,en,BLACK. LIVES. MATTER.,I haven’t talked to this girl since my sophomo...,labeled_users_1145/profile pics test/1.jpeg
2,0,___schaeffer___,Brenden Schaeffer,en,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,☝🏼👋🏼 ://t.co/7NcaO1fyc5 ://t.co/bkhrNcvp6Q 37...,labeled_users_1145/profile pics test/3.jpeg
3,0,__EmilyRice__,em,en,#TXST22,yes but come to san marcos and live with me 🥰...,labeled_users_1145/profile pics test/9.jpeg
4,0,__ginaaaa__,Gina Marano,en,WVU Nursing  20,small :).Go get ready for dinner. JACK are ...,labeled_users_1145/profile pics test/10.jpeg
...,...,...,...,...,...,...,...
1036,0,Zgs_Apollo,Anthony Sharp,en,22 Youtuber & Twitch Streamer trying to live t...,"Goodnight my friends, much love.Drop a ❤️ if t...",labeled_users_1145/profile pics test/3266.jpeg
1037,0,zmeadows_18,Z Meadows,und,|OUCÈ23__|,ROLL BOBBIES ROLL💚🖤💚🖤💚.We Are Texans! Im takin...,labeled_users_1145/profile pics test/3268.jpeg
1038,0,ZoeCalamaco,Zoe _,no,Angelo state,one person followed me // automatically checke...,labeled_users_1145/profile pics test/3272.jpeg
1039,1,ZoPeachy,Zobella Thee Alpha __ê_____´Ù__ ...,en,Harlot for hire. FinDom. 27. Nonbinary. they/t...,Good morning! Say it back ♡ Friday! Send to p...,labeled_users_1145/profile pics test/3274.jpeg


In [61]:
new_dataset_1145.shape

(1041, 7)

In [62]:
new_dataset_1145['age'].value_counts()

0    729
1    312
Name: age, dtype: int64

# 3. Race Prediction

## 3.1 Naive Bayes

In [15]:
train_data = new_dataset.sample(frac=0.8, random_state=42)
test_data = new_dataset.drop(train_data.index)

In [16]:
# train_data = augment_data(train_data)
# new_dataset = augment_data(new_dataset)

In [17]:
train_data.shape

(2501, 8)

In [18]:
test_data.shape

(625, 8)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [20]:
def trans_data(train_text, test_text, Vectorizer, max_features):
    vectorizer = Vectorizer(stop_words="english", max_features = max_features)
    vectorizer.fit(train_text)
    x_input = vectorizer.transform(train_text)
    if test_text is not None:
        x_test_input = vectorizer.transform(test_text)
    else:
        x_test_input = None
    return x_input, x_test_input

In [21]:
def predict_sentiment(train_data, test_data, Classifier, Vectorizer, label_name = 'race', max_features=None):
    train_text = train_data['tweets'].values
    test_text = test_data['tweets'].values
    x_input, x_test_input = trans_data(train_text, test_text, Vectorizer, max_features=max_features)
    clf = Classifier()
    clf.fit(x_input, train_data[label_name].values)
    y_pred = clf.predict(x_test_input)
    score = clf.score(x_test_input, test_data[label_name].values)
    # print("Classifier: {}\nFeature: {}\nscore: {}".format(Classifier.__name__, Vectorizer.__name__, score))
    print(classification_report(y_true=test_data[label_name].values, y_pred=y_pred))
    return score

In [22]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [23]:
def kfold_valid(train_data, Classifier, Vectorizer, label_name = 'race', max_features=None, scoring='accuracy'):
    train_text = train_data['tweets'].values
    x_input, _ = trans_data(train_text, None, Vectorizer, max_features=max_features)
    cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = cross_val_score(Classifier(), x_input, train_data[label_name].values, scoring=scoring, cv=cv)
    print('Classifier: {}\nFeature: {}\n{}: {}'.format(Classifier.__name__, Vectorizer.__name__, scoring, np.mean(scores)))

In [72]:
# MultinomialNB + CountVectorizer
predict_sentiment(train_data, test_data, MultinomialNB, CountVectorizer, label_name='race', max_features=10000)
kfold_valid(new_dataset, MultinomialNB, CountVectorizer, label_name='race', max_features=10000)

              precision    recall  f1-score   support

           1       0.42      0.42      0.42        48
           2       0.17      0.19      0.18        37
           3       0.06      0.05      0.06        19
           4       0.88      0.88      0.88       506

    accuracy                           0.77       610
   macro avg       0.38      0.38      0.38       610
weighted avg       0.78      0.77      0.77       610

Classifier: MultinomialNB
Feature: CountVectorizer
accuracy: 0.7276236215824635


In [74]:
# MultinomialNB + TfidfVectorizer
predict_sentiment(train_data, test_data, MultinomialNB, TfidfVectorizer, label_name='race', max_features=1000)
kfold_valid(new_dataset, MultinomialNB, TfidfVectorizer, label_name='race', max_features=1000)

E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.50      0.02      0.04        48
           2       0.44      0.11      0.17        37
           3       0.00      0.00      0.00        19
           4       0.84      0.99      0.91       506

    accuracy                           0.83       610
   macro avg       0.45      0.28      0.28       610
weighted avg       0.76      0.83      0.77       610

Classifier: MultinomialNB
Feature: TfidfVectorizer
accuracy: 0.8069469560784525


## 3.2 Logistic Regression Classifier

In [75]:
from sklearn.linear_model import LogisticRegression

In [76]:
# LogisticRegression + CountVectorizer
predict_sentiment(train_data, test_data, LogisticRegression, CountVectorizer, label_name='race', max_features=10000)
kfold_valid(new_dataset, LogisticRegression, CountVectorizer, label_name='race', max_features=10000)

E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           1       0.45      0.48      0.46        48
           2       0.10      0.05      0.07        37
           3       0.00      0.00      0.00        19
           4       0.87      0.92      0.89       506

    accuracy                           0.80       610
   macro avg       0.36      0.36      0.36       610
weighted avg       0.76      0.80      0.78       610



E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

Classifier: LogisticRegression
Feature: CountVectorizer
accuracy: 0.7967819484317566


E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [80]:
# LogisticRegression + TfidfVectorizer
predict_sentiment(train_data, test_data, LogisticRegression, TfidfVectorizer, label_name='race', max_features=1000)
kfold_valid(new_dataset, LogisticRegression, TfidfVectorizer, label_name='race', max_features=1000)

E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.57      0.08      0.15        48
           2       0.00      0.00      0.00        37
           3       0.00      0.00      0.00        19
           4       0.84      1.00      0.91       506

    accuracy                           0.83       610
   macro avg       0.35      0.27      0.26       610
weighted avg       0.74      0.83      0.77       610

Classifier: LogisticRegression
Feature: TfidfVectorizer
accuracy: 0.8230087735773122


# 4. age prediction

In [81]:
train_data_1145 = new_dataset_1145.sample(frac=0.8)
test_data_1145 = new_dataset_1145.drop(train_data_1145.index)

In [82]:
train_data_1145

,age,screen_name,name,lang,description,tweets,img_path
507,0,Jordyn_Lubeach,Jordyn Lubeach,en,UC  21 __,pls slap her again for me. U good bro?. You’...,labeled_users_1145/profile pics test/1591.jpeg
130,0,basically_adri,adri È,en,adriana ¢ T1D ¢ 16 ¢ she/her ¢ #blackl...,need to know by doja cat is just so good. y’a...,labeled_users_1145/profile pics test/391.jpeg
594,0,Lil_Kermy,Tyler Grady,en,FHS '17 Unity '21,So you’ve chosen death… Their name shall be ...,labeled_users_1145/profile pics test/1884.jpeg
1,0,___aleia,_æ___ dad ___æ_,en,BLACK. LIVES. MATTER.,I haven’t talked to this girl since my sophomo...,labeled_users_1145/profile pics test/1.jpeg
903,0,swavey_orlando,Swavey_êç,en,Sacramento State _'21. Filmoe_ HEAVY__...,Lmao I was gettin some shots today and tha dud...,labeled_users_1145/profile pics test/2864.jpeg
...,...,...,...,...,...,...,...
610,0,Lopthien,LopeRizzle,es,Not as Crippled at the Moment,I think I’d be a lot happier if o had a pet bi...,labeled_users_1145/profile pics test/1932.jpeg
68,1,alpalmarie,allie,en,BLM ___________è,😳🤯 road game since the playoff run in 2019 🙌...,labeled_users_1145/profile pics test/198.jpeg
715,1,NewfoundNolan,Miklo,en,"Been in this shit since '92, look at all the b...",I burn 6k a month just to live a basic life in...,labeled_users_1145/profile pics test/2263.jpeg
190,0,callienicole16,Californication_,en,TXST È22__,Happy birthday !!! Looks like i’ll have to buy...,labeled_users_1145/profile pics test/588.jpeg


In [83]:
test_data_1145

,age,screen_name,name,lang,description,tweets,img_path
2,0,___schaeffer___,Brenden Schaeffer,en,Culver-Stockton College '20 ¢ Ô_Ô_Ô KM 1548...,☝🏼👋🏼 ://t.co/7NcaO1fyc5 ://t.co/bkhrNcvp6Q 37...,labeled_users_1145/profile pics test/3.jpeg
13,0,_chiara475,_«_c,en,17 | still waiting for g-eazy to text me back,hit the link and donate and i might share the ...,labeled_users_1145/profile pics test/27.jpeg
14,1,_crayolacrayon,permie sanders,en,rinse. recycle. redouche. 24 | they/them | È...,"I think I’m dead, cause I can feel ghosts and...",labeled_users_1145/profile pics test/30.jpeg
21,1,_Gavin0304,Gavin,en,24. Dad. SG. LLXP.,Feels good making a list of goals and crossing...,labeled_users_1145/profile pics test/43.jpeg
25,0,_jaydezha,ur dad.,en,21 and i overthink like SHIT | NRCMA_©__...,🤣🤣🤣🤣 🤣🤣🤣 fr i wanna know 🤣🤣 said to be a s...,labeled_users_1145/profile pics test/53.jpeg
...,...,...,...,...,...,...,...
1024,0,Yalocaldoughboy,Noah6´ÙÛ£9´ÙÛ£,en,If your absence doesnÈt bother them you neve...,Idgaf who doing better than me. I’m doing bett...,labeled_users_1145/profile pics test/3233.jpeg
1025,0,Yankeluh,Yankeluh,en,"To dream the impossible dream, to fight the un...",Toll from political push at UNC continues to m...,labeled_users_1145/profile pics test/3235.jpeg
1027,0,yo_alaan,hey darren,und,big gulp slurpee straw neckass,🥴👋.Im barely keeping my eyes open 💀 someone ru...,labeled_users_1145/profile pics test/3241.jpeg
1029,1,Yoloderr,_Yoloderr,en,Kyle | @_mandapandaa_ ___ | Twitch Affiliate ...,"Thanks for hanging out, catch ya next stream!....",labeled_users_1145/profile pics test/3243.jpeg


In [84]:
# MultinomialNB + CountVectorizer
predict_sentiment(train_data_1145, test_data_1145, MultinomialNB, CountVectorizer, label_name='age', max_features=10000)
kfold_valid(new_dataset_1145, MultinomialNB, CountVectorizer, label_name='age', max_features=10000)

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       155
           1       0.48      0.28      0.36        53

    accuracy                           0.74       208
   macro avg       0.63      0.59      0.60       208
weighted avg       0.71      0.74      0.71       208

Classifier: MultinomialNB
Feature: CountVectorizer
accuracy: 0.7127668384247332


In [85]:
# MultinomialNB + TfidfVectorizer
predict_sentiment(train_data_1145, test_data_1145, MultinomialNB, TfidfVectorizer, label_name='age', max_features=2000)
kfold_valid(new_dataset_1145, MultinomialNB, TfidfVectorizer, label_name='age', max_features=2000)

              precision    recall  f1-score   support

           0       0.75      1.00      0.86       155
           1       1.00      0.02      0.04        53

    accuracy                           0.75       208
   macro avg       0.87      0.51      0.45       208
weighted avg       0.81      0.75      0.65       208

Classifier: MultinomialNB
Feature: TfidfVectorizer
accuracy: 0.7118283032756717


In [86]:
# LogisticRegression + CountVectorizer
predict_sentiment(train_data_1145, test_data_1145, LogisticRegression, CountVectorizer, label_name='age', max_features=10000)
kfold_valid(new_dataset_1145, LogisticRegression, CountVectorizer, label_name='age', max_features=10000)

E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.76      0.85      0.80       155
           1       0.34      0.23      0.27        53

    accuracy                           0.69       208
   macro avg       0.55      0.54      0.54       208
weighted avg       0.66      0.69      0.67       208



E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

Classifier: LogisticRegression
Feature: CountVectorizer
accuracy: 0.7031698564593302


E:\Applications\anaconda3\envs\cs640\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [87]:
# LogisticRegression + TfidfVectorizer
predict_sentiment(train_data_1145, test_data_1145, LogisticRegression, TfidfVectorizer, label_name='age', max_features=2000)
kfold_valid(new_dataset_1145, LogisticRegression, TfidfVectorizer, label_name='age', max_features=2000)

              precision    recall  f1-score   support

           0       0.76      1.00      0.87       155
           1       1.00      0.09      0.17        53

    accuracy                           0.77       208
   macro avg       0.88      0.55      0.52       208
weighted avg       0.82      0.77      0.69       208

Classifier: LogisticRegression
Feature: TfidfVectorizer
accuracy: 0.7118052999631946
